## Project 3: RNN

In [8]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

### Step 1: Import text

In [9]:
text = open('obama.txt',encoding='utf8').read().lower()
print('Text Length:', len(text))

Text Length: 4224142


#### Basic configurations

In [10]:
chars = sorted(list(set(text)))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Cut the text into small sentences with fixed length
txtlen = 100
step = 10
sentences = []
next_chars = []
for i in range(0, len(text) - txtlen, step):
    sentences.append(text[i: i + txtlen])
    next_chars.append(text[i + txtlen])

print('Total Chars: ', len(chars))
print('Total Sentences: ', len(sentences))

Total Chars:  94
Total Sentences:  422405


### Step 2: Vectorization

In [11]:
x_train = np.zeros((len(sentences), txtlen, len(chars)), dtype=np.bool)
y_train = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x_train[i, t, char_indices[char]] = 1
    y_train[i, char_indices[next_chars[i]]] = 1

### Step 3: Setup the model

In [12]:
model = Sequential()

# LSTM
model.add(LSTM(128, input_shape=(txtlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

rms = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=rms)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               114176    
_________________________________________________________________
dense_2 (Dense)              (None, 94)                12126     
_________________________________________________________________
activation_2 (Activation)    (None, 94)                0         
Total params: 126,302
Trainable params: 126,302
Non-trainable params: 0
_________________________________________________________________


### Step 4: Let's make some predictions

In [13]:
def sample(preds, diversity=1.0):
    preds = np.asarray(preds).astype('float64')
    log_preds = np.log(preds) / diversity
    exp_preds = np.exp(log_preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [14]:
batch_size = 1024
epochs = 1

diversity = 1.0

for i in range(1,101):
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
    start_point = random.randint(0, len(text) - txtlen - 1)
    
    if i%5 == 0:
        
        generated = ''
        sentence = text[start_point : start_point + txtlen]
        generated += sentence
        print('\n---------- Generating with: ----------')
        print(generated)
        print('\n---------- Prediction: ----------')
        
        for i in range(300):
            x_pred = np.zeros((1, txtlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        
        print()
        
model.save('model.h5')

Epoch 1/1
422405/422405 [==============================] - 59s 141us/step - loss: 2.0924
Epoch 1/1
422405/422405 [==============================] - 58s 138us/step - loss: 1.5848
Epoch 1/1
422405/422405 [==============================] - 58s 137us/step - loss: 1.4511
Epoch 1/1
422405/422405 [==============================] - 58s 138us/step - loss: 1.3875
Epoch 1/1
422405/422405 [==============================] - 58s 138us/step - loss: 1.3457

---------- Generating with: ----------
nation builders.”  here in america, it’s time we treated the people who educate our children with th

---------- Prediction: ----------
em.  effiding.  it’s membe ar

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


romation for the insolace. it’s upination. on that crimom life-experts with ourselves we la-ter from vire  and 19vid.  i alliarceess the flewil somemonaly, “understand unlome, tom obli"d. states and chasies, and$ even the nopong uses. no  alinables, pulled voice.

but th
Epoch 1/1
422405/422405 [==============================] - 58s 138us/step - loss: 1.3203
Epoch 1/1
422405/422405 [==============================] - 58s 138us/step - loss: 1.2992
Epoch 1/1
422405/422405 [==============================] - 58s 138us/step - loss: 1.2814
Epoch 1/1
422405/422405 [==============================] - 59s 139us/step - loss: 1.2661
Epoch 1/1
422405/422405 [==============================] - 58s 138us/step - loss: 1.2546

---------- Generating with: ----------
economy. and that’s why i’ve directed the federal government to hire more veterans, including more t

---------- Prediction: ----------
wosely and  because his clange safey than from the moment, one of harm toot anveruning wheppen or so import

uide these fair.  they also healthy welce. provided.  but when i'm going to safety an europe can said, and singy iraq we engaging brairm us in the path, and i spend the sanced is to strengthen john is the needic.  and so that i was important to seek his iels of americans that can pake; gives.

onlat
Epoch 1/1
422405/422405 [==============================] - 58s 138us/step - loss: 1.1492
Epoch 1/1
422405/422405 [==============================] - 58s 138us/step - loss: 1.1474
Epoch 1/1
422405/422405 [==============================] - 58s 138us/step - loss: 1.1473
Epoch 1/1
422405/422405 [==============================] - 59s 139us/step - loss: 1.1464
Epoch 1/1
422405/422405 [==============================] - 59s 139us/step - loss: 1.1454

---------- Generating with: ----------
pass health care right now, a plan that guarantees insurance to every american who wants it and brin

---------- Prediction: ----------
ks to acknowled to businest a fact that our dream measure of as entire natrace

 you can be taking this happens in the mallear of the jobs work.

this's 't this country here, something come home to choice in iraq, in a globe to the economic phone systamiests interest fody. this ceilitor puth.

if was long acsobs election that we know we know we phy rown by the world have been u
Epoch 1/1
422405/422405 [==============================] - 58s 138us/step - loss: 1.1234
Epoch 1/1
422405/422405 [==============================] - 58s 138us/step - loss: 1.1226
Epoch 1/1
422405/422405 [==============================] - 58s 137us/step - loss: 1.1230
Epoch 1/1
422405/422405 [==============================] - 58s 138us/step - loss: 1.1229
Epoch 1/1
422405/422405 [==============================] - 58s 138us/step - loss: 1.1226

---------- Generating with: ----------
sanctioned by law or by custom.  and before the civil rights movement, it most surely was.

we do a 

---------- Prediction: ----------
lot of energy isirment, and someone.  and if we found the 20throaders have, fo